In [24]:
#import necessary packages

import pandas as pd
import numpy as np
import category_encoders as ce
from category_encoders import WOEEncoder
import xgboost as xgb
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier,ExtraTreesClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix
from imxgboost.imbalance_xgb import imbalance_xgboost as imb_xgb

import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
%matplotlib inline

from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import KBinsDiscretizer, FunctionTransformer
from sklearn.model_selection import cross_val_score
from category_encoders import OneHotEncoder
from sklearn.pipeline import make_pipeline, Pipeline

from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import roc_auc_score, accuracy_score

from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split, GridSearchCV
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials

rng = np.random.RandomState(42)

import warnings
warnings.filterwarnings("ignore")

In [25]:
#read in all of the data and combine them into one dataframe called train

In [26]:
rtrain = pd.read_csv('m_race_hist.csv')

In [27]:
itrain = pd.read_csv('m_horse_info.csv')

In [28]:
train = rtrain.join(itrain.set_index('hname'), on='hname')

In [29]:
#create a target variable

In [30]:
target = train['Pla.']

In [31]:
train.drop('Pla.', axis = 1, inplace = True)

In [32]:
target = target.astype(int)

In [33]:
target = target.where(target > 3, 1)

In [34]:
target = target.where(target <= 3, 0)

In [35]:
target.value_counts()

0    33083
1    11161
Name: Pla., dtype: int64

In [36]:
#Target encode all of the categorical variables

In [37]:
used_cols = [c for c in train.columns.tolist() if c not in ['Date','Rtg.','Finish Time',
                                                            'LBW','Race  Index',
                                                           'Current Rating',
                                                            'Running  Position','Win Odds']]
X, y = train[used_cols], target

In [40]:
X.columns

In [39]:
cate = [c for c in X.columns.tolist() if c not in ['Act.  Wt.','Declar.  Horse Wt.',
                                                    'Dist.','Dr','Race Class']]

In [41]:
ce_target_encoder = ce.TargetEncoder(cols = cate, smoothing=.3)
ce_target_encoder.fit(X, target)

In [23]:
X = ce_target_encoder.transform(X)

In [406]:
#create and fit an xgb model

In [4]:
# top 3
clf = xgb.XGBClassifier(max_depth=11, n_estimators=300, colsample_bytree=0.8, 
                        subsample=0.8, nthread=10, learning_rate=.1, n_jobs = -1)
clf.fit(X, y)
# plot the important features #
fig, ax = plt.subplots(figsize=(8,10))
xgb.plot_importance(clf, max_num_features=50, height=0.8, ax=ax)
plt.show()

In [26]:
scores = cross_val_score(clf, X, y, cv=5, scoring='roc_auc')
print(scores.mean(), "+/-", scores.std())

0.8830184891289438 +/- 0.002944017036134901


In [46]:
X.sample(5)

,Act. Wt.,Course,Declar. Horse Wt.,Dist.,Dr,Jockey,RC,Race Class,Track,Trainer,hname
6430,117,C+3,1069,1000,10,K C Leung,ST,3,TURF,A T Millard,SUPREME FLIGHT
3561,133,A+3,1066,1400,4,N Callan,ST,3,Turf,D E Ferraris,LOBO'S LEGEND
40256,133,C,1137,1400,8,N Callan,ST,4,TURF,J Moore,MIDNIGHT RATTLER
41475,115,C,1047,1200,12,W M Lai,HV,2,TURF,C H Yip,JIMSON THE FAMOUS
36589,131,NaN,1007,1650,5,C Y Ho,ST,2,AWT,K W Lui,CHARLIE VICTOR
